In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.style.use('ggplot')
# https://www.kaggle.com/fedesoriano/stroke-prediction-dataset 
df = pd.read_csv("..\\..\\data\\healthcare-dataset-stroke-data.csv")

In [6]:
# Generate the data samples: Original, Oversampled, Undersampled
from src.Resample import undersample_kmeans
from src.Resample import oversample

y = df[["stroke"]].copy()
X = df.copy()
X.drop(columns= ["id", "stroke"], inplace=True)

# HANDLE NAN VALUES HERE! Add method
df.dropna(inplace=True)

# Get over and undersampled data
undersampled = undersample_kmeans(df)
oversampled = oversample(df)

y_over = oversampled[["stroke"]].copy()
y_under = undersampled[["stroke"]].copy()

oversampled.drop(columns=["stroke"], inplace=True)
undersampled.drop(columns=["stroke"], inplace=True)

X_over = oversampled.copy()
X_under = undersampled.copy()

Original dataset samples per class Counter({0: 4700, 1: 209})
Resampled dataset samples per class Counter({0: 209, 1: 209})
Original dataset samples per class Counter({0: 4700, 1: 209})
Resampled dataset samples per class Counter({1: 4700, 0: 4700})


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

num_pipe = Pipeline([
    #('impute', SimpleImputer(strategy='median', copy=False)),
    ('minmax_scaler', MinMaxScaler(copy=False))
])

ordinal_pipe = Pipeline([
    ('one_hot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

binary_pipe = Pipeline([
    ('label_encoder', OrdinalEncoder()),
])

# two pipelines combined in the column transformer
full_transform = ColumnTransformer([
    ("num", num_pipe, ["age", "avg_glucose_level", "bmi"]),
    ("ord", ordinal_pipe, ["gender", "work_type", "smoking_status"]),
    ("binary", binary_pipe, ["ever_married", "Residence_type"]),
])

full_pipeline = Pipeline([
    ('trf', full_transform),
    ('svm', SVC(random_state =0))
])

In [10]:
from sklearn.model_selection import train_test_split
# Split data

X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.33, random_state=42, shuffle=True) # Set shuffle to true to have data of both labels in test set

,age,avg_glucose_lvl,bmi,hypertension,heart_disease,gender,ever_married,work_type,Residence_type,smoking_status
0,2,80.764,22.32,0,0,Female,No,children,Urban,Unknown
1,64,95.9605,29.0575,0,0,Female,Yes,Private,Rural,never smoked
2,69,189.976111,33.033333,0,0,Female,Yes,Private,Urban,never smoked
3,38,106.088077,27.842308,0,0,Female,Yes,Private,Rural,never smoked
4,57,65.314186,30.562791,0,0,Female,Yes,Private,Rural,never smoked
...,...,...,...,...,...,...,...,...,...,...
413,68,247.51,40.5,1,1,Female,Yes,Private,Urban,formerly smoked
414,57,84.96,36.7,0,0,Male,Yes,Private,Rural,Unknown
415,14,57.93,30.9,0,0,Female,No,children,Rural,Unknown
416,75,78.8,29.3,0,0,Female,Yes,Self-employed,Rural,formerly smoked


In [11]:
full_pipeline.fit(X_train, y_train)
full_pipeline.score(X_test, y_test)

ValueError: A given column is not a column of the dataframe